In [ ]:
import matplotlib.pyplot as plt
import json, glob
import numpy as np


In [ ]:
datadest = 'C:/Users/azton/Projects/phoenix_analysis/enriched_stats'
sim = 'phx256-1'
simdicts = glob.glob('%s/*_%s.json'%(datadest, sim))
with open(simdicts[0], 'r') as f:
    data = json.load(f)
for i, file in enumerate(simdicts):
    if i == 0: continue
    with open(file, 'r') as f:
        ndata = json.load(f)
    for k in data.keys():
        if type(data[k]) == list:
            data[k] += ndata[k]
        else:
            for kk in data[k].keys():
                data[k][kk] += ndata[k][kk]

In [ ]:
sort_inds = np.argsort(data['t'])
for k in data.keys():
    if type(data[k]) == list:
        data[k] = [data[k][i] for i in sort_inds]
    else:
        for kk in data[k].keys():
            data[k][kk] = [data[k][kk][i] for i in sort_inds]

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
for k in data['f_enr_od']:
    ax.plot(data['z'], data['f_enr_od'][k], label = '$\\rho/\\bar\\rho > $%s'%k if k != '<1' else '$\\rho/\\bar\\rho %s$'%k)
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel("F$_{enriched}$")
ax.set_xlabel('Lookback Time [Myr]')
ax.set_yscale('log')
plt.savefig('enriched_stats/f_enrich-overdense.pdf', bbox_inches='tight')


In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
time = [max(data['t'])-t for t in data['t']]
for k in data['f_enr_high']:
    ax.plot(time, data['f_enr_high'][k], label = '$Z_c = $%0.2e'%float(k))
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel("F$_{enriched}$")
ax.set_xlabel('Lookback Time [Myr]')
ax.set_yscale('log')
plt.savefig('enriched_stats/f_enrich-zdist.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
# zdiff = [ data['p3_f_enr'][j] - (data['f_enr'][j] -data['p3_f_enr'][j]) for j in range(len(data['f_enr']))]
ax.plot(data['z'], data['p3_f_enr'], label='$Z_{III}$')
ax.plot(data['z'], data['f_enr'], label='$Z_\\sum$')
ax.legend()
ax.set_ylabel("f$_{enriched}(Z > Z_c)$")
ax.set_xlabel('Lookback Time [Myr]')
ax.set_yscale('log')
plt.savefig('enriched_stats/f_enrich-p3-psum.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(3,4))
for k in data['ion_frac']:
    if np.all(np.array(data['ion_frac'][k]) < 1):
        ax.plot(data['z'], data['ion_frac'][k], label = '$f_{\\rm ion} > $%0.3f'%(float(k)))
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel("F$_{ionized}$")
ax.set_xlabel('Lookback Time [Myr]')
ax.set_yscale('log')
plt.savefig('enriched_stats/f_ionized.pdf', bbox_inches='tight')